# Custom administrative codes static time series

In this notebook, we create static time series representing the custom codes to assign to each administrative region of our countries. The identification code is assigned without resetting it from one country to another.

In [1]:
import geopandas as gpd
import pandas as pd
import glob

In [2]:
# Add the python path to the folder containing some custom packages.
import sys
sys.path.insert(0, "../../packages/")
from TsIP.TsIP import TsIP

In [3]:
# Define the path where the results arising from this analysis will be saved.
PATH_TO_SAVE_DATA = "./output_timeseries/"

In [4]:
# Read the shapefiles of each country to get the adminstrata information.
path = "../../packages/MapPlot/shapefiles/adminstratas/"
all_folders = glob.glob(path + "*")

dfs = []

for folder in all_folders:
    all_files = glob.glob(folder + "/*.shp")
    for filename in all_files:
        df = gpd.read_file(filename)
        dfs.append(df)

In [5]:
# Concatenate the data.
df = pd.concat(dfs, axis = 0, ignore_index = True)
df.head()

,country,region,geometry
0,Burkina Faso,Boucle-Du-Mouhoun,"POLYGON ((-3.18158 13.67744, -3.18109 13.67721..."
1,Burkina Faso,Cascades,"POLYGON ((-5.38923 11.00086, -5.37382 10.97698..."
2,Burkina Faso,Centre,"POLYGON ((-1.60508 12.67250, -1.58739 12.67057..."
3,Burkina Faso,Centre-Est,"POLYGON ((-0.30646 12.55612, -0.30573 12.55588..."
4,Burkina Faso,Centre-Nord,"MULTIPOLYGON (((-0.76975 12.55062, -0.77002 12..."


In [6]:
# Adjust the dataframe.
df.reset_index(inplace = True)
df.rename(columns = {"region": "AdminStrata", "country": "Country", "index": "Code"}, inplace = True)
df.drop(columns = "geometry", inplace = True)
df.head()

,Code,Country,AdminStrata
0,0,Burkina Faso,Boucle-Du-Mouhoun
1,1,Burkina Faso,Cascades
2,2,Burkina Faso,Centre
3,3,Burkina Faso,Centre-Est
4,4,Burkina Faso,Centre-Nord


In [7]:
# Create multi-index columns dataframe.
df = df.set_index(["Country", "AdminStrata"]).T.reset_index(drop = True)
df.columns = pd.MultiIndex.from_tuples(list(map(lambda x: tuple(list(x) + ["Code"]), df.columns)), 
                                       names = ["Country", "AdminStrata", "Indicator"])
df

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                Code     Code   Code       Code        Code   
0                           0        1      2          3           4   

Country                                                      ...  Yemen       \
AdminStrata Centre-Ouest Centre-Sud  Est Hauts-Bassins Nord  ... Sana'a Aden   
Indicator           Code       Code Code          Code Code  ...   Code Code   
0                      5          6    7             8    9  ...    103  104   

Country                                                                      
AdminStrata Lahj Marib Al Mahwit Al Maharah Amran Al Dhale'e Raymah Socotra  
Indicator   Code  Code      Code       Code  Code       Code   Code    Code  
0            105   106       107        108   109        110    111     112  

[1 rows x 113 columns]

In [8]:
# Define two years between which we want to build the static time series.
FIRST_YEAR = "2018"
LAST_YEAR = "2023"

new_index = pd.date_range(FIRST_YEAR, LAST_YEAR, freq = "D")

In [9]:
# Create static time series regarding the adminstrative codes.
df = df.loc[df.index.repeat(len(new_index))]
df.index = new_index
df.index.name = "Datetime"
df.head()

Country          Burkina Faso                                         \
AdminStrata Boucle-Du-Mouhoun Cascades Centre Centre-Est Centre-Nord   
Indicator                Code     Code   Code       Code        Code   
Datetime                                                               
2018-01-01                  0        1      2          3           4   
2018-01-02                  0        1      2          3           4   
2018-01-03                  0        1      2          3           4   
2018-01-04                  0        1      2          3           4   
2018-01-05                  0        1      2          3           4   

Country                                                      ...  Yemen       \
AdminStrata Centre-Ouest Centre-Sud  Est Hauts-Bassins Nord  ... Sana'a Aden   
Indicator           Code       Code Code          Code Code  ...   Code Code   
Datetime                                                     ...               
2018-01-01             5          6    7             8    9  ...    103  104   
2018-01-02             5          6    7             8    9  ...    103  104   
2018-01-03             5          6    7             8    9  ...    103  104   
2018-01-04             5          6    7             8    9  ...    103  104   
2018-01-05             5          6    7             8    9  ...    103  104   

Country                                                                      
AdminStrata Lahj Marib Al Mahwit Al Maharah Amran Al Dhale'e Raymah Socotra  
Indicator   Code  Code      Code       Code  Code       Code   Code    Code  
Datetime                                                                     
2018-01-01   105   106       107        108   109        110    111     112  
2018-01-02   105   106       107        108   109        110    111     112  
2018-01-03   105   106       107        108   109        110    111     112  
2018-01-04   105   106       107        108   109        110    111     112  
2018-01-05   105   106       107        108   109        110    111     112  

[5 rows x 113 columns]

In [10]:
# Now save the time series of each country keeping as indeces for each time series the own first and last index.
def save(group, name):
    country = group.name
    group = group[country]
    # Adjust time series group.
    first_idx = group.first_valid_index()
    last_idx = group.last_valid_index()
    group = group.loc[first_idx:last_idx]
    # Save.
    group.to_csv(PATH_TO_SAVE_DATA + country + "/" + name + ".csv", index_label = False)

In [11]:
# Save the static time series.
df.groupby(level = "Country", axis = 1).apply(lambda x: save(x, name = "wfp_adminstrata_code"));

In [12]:
# Plot static time series.
TsIP(df).interactive_plot_df(matplotlib = False, style = "lines", yaxis = "Adminstrata code")

interactive(children=(ToggleButtons(description='Country', options=('Burkina Faso', 'Cameroon', 'Mali', 'Niger…